In [ ]:
from source.source.postprocessing_utils import (
    get_sampled_combinations_uncertainty_scores,
    get_predicted_labels,
    get_missclassification_dataframe,
    get_ood_detection_dataframe,
    get_raw_scores_dataframe,
)

from source.datasets.constants import DatasetName
from source.losses.constants import LossName
from source.models.constants import ModelName
from source.metrics import (
    ApproximationType,
    GName,
    RiskType,
)

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)

stty: 'standard input': Inappropriate ioctl for device


In [ ]:
training_dataset_names = [
    "cifar10",
    "cifar100",
    "noisy_cifar100",
    "noisy_cifar10",
]
temperature = 1.0
model_ids = np.arange(20)

list_extraction_datasets = [
    "cifar10",
    "cifar100",
    "svhn",
    "blurred_cifar100",
    "blurred_cifar10",
]
list_ood_datasets = [el for el in list_extraction_datasets]
loss_function_names = [el for el in LossName]

full_dataframe = None
full_ood_rocauc_dataframe = None
full_mis_rocauc_dataframe = None

In [3]:
for training_dataset_name in training_dataset_names:
    architectures = [ModelName.RESNET18, ModelName.VGG19]
    if training_dataset_name not in [
        "noisy_cifar10",
        "noisy_cifar100",
    ]:
        training_dataset_name_aux = training_dataset_name
    else:
        # architectures = [ModelName.RESNET18]
        training_dataset_name_aux = training_dataset_name.split("_")[-1]
    for architecture in architectures:
        # try:
        uq_results, embeddings_per_dataset, targets_per_dataset = (
            get_sampled_combinations_uncertainty_scores(
                loss_function_names=loss_function_names,
                training_dataset_name=training_dataset_name,
                architecture=architecture,
                model_ids=model_ids,
                list_extraction_datasets=list_extraction_datasets,
                temperature=temperature,
                use_cached=True,
            )
        )

        df_ood = get_ood_detection_dataframe(
            ind_dataset=training_dataset_name_aux,
            uq_results=uq_results,
            list_ood_datasets=list_ood_datasets,
        )

        max_ind = int(
            targets_per_dataset[training_dataset_name_aux].shape[0] / len(model_ids)
        )
        true_labels = targets_per_dataset[training_dataset_name_aux][:max_ind]

        pred_labels = get_predicted_labels(
            embeddings_per_dataset=embeddings_per_dataset,
            training_dataset_name=training_dataset_name_aux,
        )

        df_misclassification = get_missclassification_dataframe(
            ind_dataset=training_dataset_name_aux,
            uq_results=uq_results,
            true_labels=true_labels,
            pred_labels=pred_labels,
        )

        scores_df_unravel = get_raw_scores_dataframe(uq_results=uq_results)
        scores_df_unravel["architecture"] = architecture.value
        scores_df_unravel["training_dataset"] = training_dataset_name
        df_ood["architecture"] = architecture.value
        df_ood["training_dataset"] = training_dataset_name
        df_misclassification["architecture"] = architecture.value
        df_misclassification["training_dataset"] = training_dataset_name

        if full_dataframe is None:
            full_dataframe = scores_df_unravel
            full_ood_rocauc_dataframe = df_ood
            full_mis_rocauc_dataframe = df_misclassification
        else:
            full_dataframe = pd.concat([full_dataframe, scores_df_unravel])
            full_ood_rocauc_dataframe = pd.concat([full_ood_rocauc_dataframe, df_ood])
            full_mis_rocauc_dataframe = pd.concat(
                [full_mis_rocauc_dataframe, df_misclassification]
            )

In [4]:
full_dataframe.sample(10)

,UQMetric,LossFunction,Dataset,Scores,architecture,training_dataset
1034,SphericalScore TotalRisk inner central,SphericalScore,blurred_cifar10,"[[0.8511269308342937, 0.8410108471819411, 0.56...",resnet18,noisy_cifar100
1009,SphericalScore TotalRisk inner inner,CrossEntropy,blurred_cifar10,"[[0.6356498, 0.7770459, 0.47114694, 0.7784482,...",resnet18,cifar100
1099,SphericalScore ExcessRisk outer inner,CrossEntropy,blurred_cifar10,"[[0.35271794, 0.24472526, 0.36817816, 0.199850...",vgg19,cifar100
1034,SphericalScore TotalRisk inner central,SphericalScore,blurred_cifar10,"[[0.33558617309339317, 0.3398629691544145, 0.3...",vgg19,noisy_cifar10
352,BrierScore TotalRisk outer central,BrierScore,svhn,"[[0.93739754, 0.51048845, 0.663387, 0.974157, ...",resnet18,noisy_cifar100
809,ZeroOneScore ExcessRisk outer central,SphericalScore,blurred_cifar10,"[[0.06477745, 0.037318498, 0.019105341, 0.0115...",resnet18,cifar100
133,LogScore TotalRisk central central,SphericalScore,blurred_cifar100,"[[4.459503, 4.102327, 4.2849994, 4.4964476, 1....",resnet18,noisy_cifar100
101,LogScore TotalRisk central outer,SphericalScore,cifar100,"[[4.54276, 3.855468, 4.175923, 3.5030837, 1.34...",resnet18,noisy_cifar100
625,BrierScore BayesRisk central,SphericalScore,cifar10,"[[0.97916573, 0.96758556, 0.98408175, 0.984525...",resnet18,noisy_cifar100
832,ZeroOneScore ExcessRisk inner inner,BrierScore,svhn,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",resnet18,cifar10


In [5]:
pattern_baserule = r"(LogScore|BrierScore|ZeroOneScore|SphericalScore)"
pattern_risk = r"(outer outer|outer inner|outer central|inner outer|inner inner|inner central|central outer|central inner|central central|energy inner|energy outer|outer|inner|central)"

full_ood_rocauc_dataframe["base_rule"] = full_ood_rocauc_dataframe[
    "UQMetric"
].str.extract(pattern_baserule)
full_ood_rocauc_dataframe["RiskType"] = full_ood_rocauc_dataframe[
    "UQMetric"
].str.extract(pattern_risk)

full_mis_rocauc_dataframe["base_rule"] = full_mis_rocauc_dataframe[
    "UQMetric"
].str.extract(pattern_baserule)
full_mis_rocauc_dataframe["RiskType"] = full_mis_rocauc_dataframe[
    "UQMetric"
].str.extract(pattern_risk)

full_dataframe["base_rule"] = full_dataframe["UQMetric"].str.extract(pattern_baserule)
full_dataframe["RiskType"] = full_dataframe["UQMetric"].str.extract(pattern_risk)

In [6]:
full_dataframe.sample(10)

,UQMetric,LossFunction,Dataset,Scores,architecture,training_dataset,base_rule,RiskType
93,LogScore TotalRisk central outer,CrossEntropy,blurred_cifar100,"[[0.011189396, 0.8426998, 0.908934, 0.7359641,...",vgg19,cifar10,LogScore,central outer
97,LogScore TotalRisk central outer,BrierScore,svhn,"[[2.8315268, 4.201318, 2.2344062, 4.2453523, 1...",vgg19,cifar100,LogScore,central outer
39,LogScore TotalRisk outer central,BrierScore,blurred_cifar10,"[[1.3873312, 1.2786677, 1.7778447, 1.3669904, ...",resnet18,noisy_cifar10,LogScore,outer central
1047,SphericalScore TotalRisk central outer,SphericalScore,svhn,"[[0.3050127089939542, 0.9316350817912455, 0.93...",vgg19,noisy_cifar100,SphericalScore,central outer
855,ZeroOneScore ExcessRisk central outer,CrossEntropy,cifar10,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",resnet18,cifar10,ZeroOneScore,central outer
262,LogScore ExcessRisk central central,BrierScore,svhn,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",vgg19,noisy_cifar100,LogScore,central central
230,LogScore ExcessRisk central outer,BrierScore,cifar10,"[[0.2652291, 0.3296075, 0.20961848, 0.19876559...",resnet18,cifar100,LogScore,central outer
38,LogScore TotalRisk outer central,BrierScore,blurred_cifar100,"[[1.041721, 1.9170747, 1.9760983, 1.4662366, 1...",vgg19,noisy_cifar10,LogScore,outer central
16,LogScore TotalRisk outer inner,CrossEntropy,cifar100,"[[0.26124758, 1.5740552, 0.8855655, 1.5968792,...",resnet18,cifar10,LogScore,outer inner
466,BrierScore ExcessRisk outer inner,CrossEntropy,cifar100,"[[0.042017482, 0.08552958, 0.07899767, 0.16964...",resnet18,noisy_cifar100,BrierScore,outer inner


In [7]:
full_ood_rocauc_dataframe.sample(10)

,UQMetric,Dataset,LossFunction,RocAucScores_array,architecture,training_dataset,base_rule,RiskType
755,ZeroOneScore TotalRisk central central,cifar100,SphericalScore,"[0.5, 0.5, 0.5, 0.49999999999999994, 0.5]",resnet18,noisy_cifar100,ZeroOneScore,central central
377,BrierScore TotalRisk inner inner,cifar10,SphericalScore,"[0.80731322, 0.8091174400000001, 0.805476205, ...",resnet18,cifar100,BrierScore,inner inner
268,LogScore ExcessRisk central central,blurred_cifar10,BrierScore,"[0.5, 0.5, 0.5, 0.5, 0.5]",resnet18,noisy_cifar10,LogScore,central central
382,BrierScore TotalRisk inner inner,svhn,BrierScore,"[0.9095203326674861, 0.8884668792255686, 0.933...",vgg19,noisy_cifar10,BrierScore,inner inner
849,ZeroOneScore ExcessRisk inner central,blurred_cifar100,CrossEntropy,"[0.5, 0.5, 0.5, 0.5, 0.5]",resnet18,noisy_cifar10,ZeroOneScore,inner central
78,LogScore TotalRisk inner central,cifar100,CrossEntropy,"[0.49999999999999994, 0.5, 0.5, 0.5, 0.4999999...",resnet18,noisy_cifar100,LogScore,inner central
167,LogScore ExcessRisk outer central,cifar10,SphericalScore,"[0.5, 0.5, 0.5, 0.5, 0.5]",resnet18,cifar10,LogScore,outer central
1217,SphericalScore BayesRisk outer,cifar10,SphericalScore,"[0.8064939100000001, 0.8083965149999999, 0.804...",resnet18,cifar100,SphericalScore,outer
696,ZeroOneScore TotalRisk inner inner,svhn,CrossEntropy,"[0.7635088372003688, 0.7749175437922557, 0.748...",vgg19,noisy_cifar100,ZeroOneScore,inner inner
771,ZeroOneScore ExcessRisk outer outer,svhn,CrossEntropy,"[0.6828574523663186, 0.6783118296711738, 0.669...",resnet18,cifar100,ZeroOneScore,outer outer


In [10]:
# full_dataframe.to_csv('../../tables/central_tables/full_dataframe.csv')
# full_ood_rocauc_dataframe.to_csv('../../tables/central_tables/full_ood_rocauc.csv')
# full_mis_rocauc_dataframe.to_csv('../../tables/central_tables/full_mis_rocauc.csv')

In [8]:
import pickle

In [9]:
full_dataframe.to_pickle("../../tables/central_tables/full_dataframe.csv")
full_ood_rocauc_dataframe.to_pickle("../../tables/central_tables/full_ood_rocauc.csv")
full_mis_rocauc_dataframe.to_pickle("../../tables/central_tables/full_mis_rocauc.csv")